In [60]:
import pandas as pd
import numpy as np
from datetime import datetime


In [61]:
df = pd.read_csv('../data/processed/cleaned_data.csv')


In [62]:
date_columns = ['Order.Date', 'Ship.Date']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"Converted {col} to datetime")

Converted Order.Date to datetime
Converted Ship.Date to datetime


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Category              51290 non-null  object        
 1   City                  51290 non-null  object        
 2   Country               51290 non-null  object        
 3   Customer.ID           51290 non-null  object        
 4   Customer.Name         51290 non-null  object        
 5   Discount              51290 non-null  float64       
 6   Market                51290 non-null  object        
 7   è®°å½æ°             51290 non-null  int64         
 8   Order.Date            51290 non-null  datetime64[ns]
 9   Order.ID              51290 non-null  object        
 10  Order.Priority        51290 non-null  object        
 11  Product.ID            51290 non-null  object        
 12  Product.Name          51290 non-null  object        
 13  Profit          

In [64]:
def generate_sales_summary(df):
    print("Generating sales summary...")
    
    sales_summary = {
        'total_sales': df['Sales'].sum(),
        'total_profit': df['Profit'].sum(),
        'average_order_value': df['Sales'].mean(),
        'profit_margin': (df['Profit'].sum() / df['Sales'].sum()) * 100,
        'total_orders': len(df),
        'unique_customers': df['Customer.ID'].nunique(),
        'date_range': f"{df['Order.Date'].min()} to {df['Order.Date'].max()}"
    }
    
    # Convert to DataFrame for easy export
    summary_df = pd.DataFrame(list(sales_summary.items()), 
                             columns=['Metric', 'Value'])
    summary_df.to_csv('../data/processed/sales_summary.csv', index=False)
    
    return summary_df

In [65]:
sales_summary = generate_sales_summary(df)

Generating sales summary...


In [66]:
def analyze_customer_segments(df):
    """Analyze customer segments and behavior"""
    
    customer_analysis = df.groupby('Customer.ID').agg({
        'Sales': ['sum', 'count', 'mean'],
        'Profit': 'sum',
        'Order.Date': ['min', 'max']
    }).round(2)
    
    # Flatten column names
    customer_analysis.columns = ['Total_Sales', 'Order_Count', 'Avg_Order_Value', 
                                'Total_Profit', 'First_Order', 'Last_Order']
    
    # Add customer segments based on sales
    customer_analysis['Customer_Segment'] = pd.cut(
        customer_analysis['Total_Sales'], 
        bins=[0, 500, 2000, float('inf')], 
        labels=['Low Value', 'Medium Value', 'High Value']
    )
    
    # Reset index to include Customer ID
    customer_analysis = customer_analysis.reset_index()
    customer_analysis.to_csv('../data/processed/customer_analysis.csv', index=False)
    
    return customer_analysis

In [67]:
customer_analysis = analyze_customer_segments(df)


In [68]:
def analyze_regional_performance(df):
    """Analyze performance by region/country"""
    
    regional_cols = ['Country', 'Region', 'State']
    available_cols = [col for col in regional_cols if col in df.columns]
    
    if not available_cols:
        print("No regional columns found")
        return None
    
    # Use the first available regional column
    region_col = available_cols[0]
    
    regional_analysis = df.groupby(region_col).agg({
        'Sales': ['sum', 'mean', 'count'],
        'Profit': ['sum', 'mean'],
        'Customer.ID': 'nunique'
    }).round(2)
    
    # Flatten column names
    regional_analysis.columns = ['Total_Sales', 'Avg_Sales', 'Order_Count', 
                                'Total_Profit', 'Avg_Profit', 'Unique_Customers']
    
    # Add profit margin
    regional_analysis['Profit_Margin'] = (
        regional_analysis['Total_Profit'] / regional_analysis['Total_Sales'] * 100
    ).round(2)
    
    # Sort by total sales
    regional_analysis = regional_analysis.sort_values('Total_Sales', ascending=False)
    regional_analysis = regional_analysis.reset_index()
    
    regional_analysis.to_csv('../data/processed/regional_analysis.csv', index=False)
    
    return regional_analysis

In [69]:
regional_analysis = analyze_regional_performance(df)


In [70]:
def analyze_product_performance(df):
    """Analyze product performance"""
    
    product_cols = ['Category', 'Sub-Category', 'Product Name']
    available_cols = [col for col in product_cols if col in df.columns]
    
    if not available_cols:
        print("No product columns found")
        return None
    
    # Analyze by Category first
    category_col = available_cols[0]
    
    product_analysis = df.groupby(category_col).agg({
        'Sales': ['sum', 'mean', 'count'],
        'Profit': ['sum', 'mean'],
        'Quantity': 'sum' if 'Quantity' in df.columns else 'count'
    }).round(2)
    
    # Flatten column names
    if 'Quantity' in df.columns:
        product_analysis.columns = ['Total_Sales', 'Avg_Sales', 'Order_Count', 
                                   'Total_Profit', 'Avg_Profit', 'Total_Quantity']
    else:
        product_analysis.columns = ['Total_Sales', 'Avg_Sales', 'Order_Count', 
                                   'Total_Profit', 'Avg_Profit', 'Total_Orders']
    
    # Add profit margin
    product_analysis['Profit_Margin'] = (
        product_analysis['Total_Profit'] / product_analysis['Total_Sales'] * 100
    ).round(2)
    
    # Sort by total sales
    product_analysis = product_analysis.sort_values('Total_Sales', ascending=False)
    product_analysis = product_analysis.reset_index()
    
    product_analysis.to_csv('../data/processed/product_performance.csv', index=False)
    
    return product_analysis

In [71]:
product_analysis = analyze_product_performance(df)


In [72]:
def analyze_monthly_trends(df):
    """Analyze monthly sales trends"""
    
    # Create year-month column
    df['Year_Month'] = df['Order.Date'].dt.to_period('M')
    
    monthly_trends = df.groupby('Year_Month').agg({
        'Sales': ['sum', 'mean', 'count'],
        'Profit': ['sum', 'mean'],
        'Customer.ID': 'nunique'
    }).round(2)
    
    # Flatten column names
    monthly_trends.columns = ['Total_Sales', 'Avg_Sales', 'Order_Count', 
                             'Total_Profit', 'Avg_Profit', 'Unique_Customers']
    
    # Add profit margin
    monthly_trends['Profit_Margin'] = (
        monthly_trends['Total_Profit'] / monthly_trends['Total_Sales'] * 100
    ).round(2)
    
    # Reset index
    monthly_trends = monthly_trends.reset_index()
    monthly_trends['Year_Month'] = monthly_trends['Year_Month'].astype(str)
    
    monthly_trends.to_csv('../data/processed/monthly_trend.csv', index=False)
    
    return monthly_trends

In [73]:
monthly_trends = analyze_monthly_trends(df)
